# Intelligente Agenten
Beantworten Sie die folgenden Fragen!
## Der Turing-Test
1. Wie funktioniert der Turing-Test?
2. Was braucht eine Maschine um den Test zu bestehen?
3. Was ist der totale Turing-Test?
4. Was braucht eine Maschine um den totalen Turing-Test zu bestehen?
5. Welche Probleme existieren beim Turing-Test?

## Agenten
1. Was ist ein Agent? Aus welchen Bestandteilen besteht ein Agent?
2. Welche Arten von Agenten kennen Sie? Wie zeichnen Sich die einzelnen Agententypen aus?
3. Durch welche Eigenschaften zeichnet sich eine Umgebung aus in der sich ein Agent bewegt?:<br/>  

## Der Staubsauger als Agent
Wir werden heute einen Staubsauger-Agenten und die dazugehörige Umgebung bzw. "Welt" programmieren.

In [ ]:
import random

Zunächst definieren wir uns eine Klasse "Environment". Diese Klasse repräsentiert unsere Welt in der sich der Agent bewegen soll.

In [ ]:
class Environment:
    """Class representing an Environment. Other environment classes can
    inherit from this. They typically need to implement:
        percept:           Define the percept that an agent sees.
        execute_action:    Define the effects of executing an action.
                           Also update the agent.performance slot.
    The environment keeps a dict of .loc_status which includes locations and their respective status. 
    There is also a list of .agents.
    """

    def __init__(self):
        
        self.loc_status = {(0, 0): random.choice(['Clean', 'Dirty']),
                          (0, 1): random.choice(['Clean', 'Dirty']),
                          (1, 0): random.choice(['Clean', 'Dirty']),
                          (1, 1): random.choice(['Clean', 'Dirty'])}
        self.agents = []
        self.time = 0


    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        return (agent.location, self.loc_status[agent.location])

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status"""
        former_location = agent.location
        if action == 'move_left' and agent.location[1] > 0:
            agent.location = (agent.location[0], agent.location[1]-1)
            print(f"Moving left: {former_location} -> {agent.location}")
            print("World state: ", self.loc_status)
        elif action == 'move_right' and agent.location[1] < 1:
            agent.location = (agent.location[0], agent.location[1]+1)
            print(f"Moving right: {former_location} -> {agent.location}")
            print("World state: ", self.loc_status)
        elif action == 'move_down' and agent.location[0] < 1:
            agent.location = (agent.location[0]+1, agent.location[1])
            print(f"Moving down: {former_location} -> {agent.location}")
            print("World state: ", self.loc_status)
        elif action == 'move_up' and agent.location[0] > 0:
            agent.location = (agent.location[0]-1, agent.location[1])
            print(f"Moving up: {former_location} -> {agent.location}")
            print("World state: ", self.loc_status)
        elif action == 'clean':
            self.loc_status[agent.location] = 'Clean'
            print("Cleaning: ", agent.location)
            print("World state: ", self.loc_status)
        elif action == 'do_nothing':
            print("Doing nothing at location:", agent.location)
            print("World state: ", self.loc_status)
        else: 
            print(f"Invalid action {action} at location ", agent.location)
    

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([(0,0), (0,1), (1,0), (1,1)])

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        actions = []
        
        for agent in self.agents:
            actions.append(agent.program(self.percept(agent)))
        for (agent, action) in zip(self.agents, actions):
            self.execute_action(agent, action)
        self.exogenous_change()

    def run(self, steps=10):
        """Run the Environment for given number of time steps."""
        print("Initial world state: ", self.loc_status)
        for step in range(steps):
            self.time += 1
            self.step()
            if all(value == "Clean" for value in self.loc_status.values()):
                break

    def add_agent(self, agent, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if agent in self.agents:
            print("Can't add the same agent twice")
        else:
            agent.location = location if location is not None else self.default_location(agent)
            self.agents.append(agent)

    def delete_agent(self, agent):
        """Remove agent from the environment."""
        try:
            self.agents.remove(agent)
        except ValueError as e:
            print(e)
            print("  in Environment delete_agent")
            print("  Agent to be removed: {} at {}".format(agent, agent.location))
            print("  from list: {}".format([(agent, agent.location) for agent in self.agents]))

Dann benötigen wir noch eine Klasse für unsere Agenten. Da wir im Laufe der Übung unterschiedliche Agenten implementieren möchten verwenden wir zunächst eine sogenannte "abstrakte" Klasse `TraceAgent`, von der unsere späteren konkreten Agent-Klassen erben können.

In [ ]:
class TraceAgent:
    """An abstract class of an agent. Concrete agent classes wil inherit from this class and have to provide (at least) a program function.
    Optionally you can also provide an __init__ function if the agent should have internal states."""
    def __init__(self):
        self.loc_status = None
        self.location = None
        
    def programm(self):
        pass

Hier sehen wir nun eine mögliche Implementierung einer `SimpleAgent`-Klasse. Dabei erbt diese Klasse alle Methoden und Attribute von der abstrakten Klasse `TraceAgent` und überschreibt diese gegebenenfalls mit ihrer eigenen Logik. 

In [ ]:
class SimpleAgent(TraceAgent):
    def program(self, perception):
        self.location, loc_status = perception
        if loc_status == "Clean":
            action = "do_nothing"
        else:
            action = "clean"
        return action

Nun könnne wir uns mit folgender Zelle eine neue Welt erstellen und unseren `SimpleAgent`zu dieser Welt hinzufügen. Anschließend Simulieren wir 10 Zeitschritte in dieser Welt.

In [ ]:
e = Environment()
e.add_agent(SimpleAgent())
e.run(10)

## Aufgabe 1
Implementieren Sie die Klasse ReflexAgent als einfachen Reflex-Agenten. Das Verhalten können sie dabei selbst festlegen. Der Agent soll dabei in unserer Welt "Environment" funktionieren. Beachten Sie diesen Umstand bei der Festlegung möglicher Aktionen, die der Agent in der genannten Welt ausführen soll.

In [ ]:
class ReflexAgent(TraceAgent):
    def program(self, perception):
        self.location, loc_status = perception
        return "Implement me!"

e1 = Environment()
a1 = ReflexAgent()
e1.add_agent(a1)
e1.run()

## Aufgabe 2
Implementieren Sie die Klasse "ModelBasedAgent" als modellbasierten Agenten. Auch dieser Agent soll in der "Environment" funktionieren. 

In [ ]:
class ModelBasedAgent(TraceAgent):
    def __init__(self):
        self.loc_status = "Implement me!"
        self.location = None
    def program(self, perception):
        location, loc_status = perception
        self.location = location
        self.loc_status[self.location] = loc_status
        
        return "Implement my logic!"

e2 = Environment()
a2 = ModelBasedAgent()
e2.add_agent(a2)
e2.run()

## Aufgabe 3
Erweitern Sie die Implementierung der Environment, sodass es beliebig viele, auf einem 2D-Gitter angeordnete Positionen gibt. Erweitern Sie auch die Implementierung der beiden Agenten entsprechend, sodass diese sinnvoll in der neuen Umgebung agieren können

In [ ]:
class ExtendedEnvironment(Environment):

    """This environment has nxn locations. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status. This serves as an example of how to implement a simple
    Environment."""

    def __init__(self, grid_size: int):
        super().__init__()
        "Implement me!"

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status"""
        
        "Implement me!"

In [ ]:
class ExtendedModelBasedAgent(TraceAgent):
    def __init__(self, loc_status):
        self.loc_status = "Implement me!"
        self.location = None
    def program(self, perception):
        location, loc_status = perception
        self.location = location
        self.loc_status[self.location] = loc_status
        
        return "Implement my logic!"

e4 = Environment()
a4 = ExtendedModelBasedAgent(e4.loc_status)
e4.add_agent(a4)
e4.run()